In [ ]:
# =====================================================
# 1️⃣ Imports
# =====================================================
import os
import urllib.request
import numpy as np
from tqdm import tqdm
from torch import nn, optim
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# =====================================================
# 2️⃣ Configurations
# =====================================================
DATA_DIR = "quickdraw_npy"
SAMPLES_PER_CLASS = 2000
IMG_SIZE = 28
BATCH_SIZE = 64
EPOCHS = 50
PATIENCE = 7
TARGET_LOSS = 0.01

# ✅ Categories (51)
categories = [
    "airplane","ant","apple","backpack","banana","bed","bicycle","bird","book","bread",
    "bus","cake","camera","car","cat","chair","clock","computer","cup","dog","door",
    "duck","envelope","eye","face","fish","flower","fork","frog","guitar","hat","horse",
    "house","lollipop","pencil","pig","pizza","rabbit","shoe","snake","spider","spoon",
    "star","sun","train","tree","truck","umbrella","alarm_clock","birthday_cake","butterfly"
]
print("📦 Total categories:", len(categories))

os.makedirs(DATA_DIR, exist_ok=True)

# =====================================================
# 3️⃣ Download .npy Files with Error Handling
# =====================================================
def download_drawings():
    base_url = "https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap"
    failed = []
    for cat in tqdm(categories, desc="⏬ Downloading categories"):
        file_path = os.path.join(DATA_DIR, f"{cat}.npy")
        if not os.path.exists(file_path):
            url = f"{base_url}/{cat.replace('_', '%20')}.npy"  # Handle underscores
            try:
                urllib.request.urlretrieve(url, file_path)
            except Exception as e:
                print(f"❌ Failed to download {cat}: {e}")
                failed.append(cat)
    print("✅ Download complete!")
    if failed:
        print(f"⚠️ Failed downloads: {failed}")

download_drawings()

# =====================================================
# 4️⃣ Dataset Class
# =====================================================
class SketchRNNDataset(Dataset):
    def __init__(self, root_dir, categories, samples_per_class):
        self.data, self.labels = [], []
        for label, cat in enumerate(categories):
            file_path = os.path.join(root_dir, f"{cat}.npy")
            if not os.path.exists(file_path):
                continue  # Skip missing files
            drawings = np.load(file_path)
            drawings = drawings.reshape(-1, IMG_SIZE, IMG_SIZE)
            if len(drawings) < samples_per_class:
                continue
            idx = np.random.choice(len(drawings), samples_per_class, replace=False)
            self.data.append(drawings[idx])
            self.labels.extend([label] * samples_per_class)
        self.data = np.vstack(self.data).astype(np.float32) / 255.0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx]).unsqueeze(0)  # (1, 28, 28)
        y = self.labels[idx]
        return x, y

# =====================================================
# 5️⃣ Prepare Dataset
# =====================================================
dataset = SketchRNNDataset(DATA_DIR, categories, SAMPLES_PER_CLASS)
train_size = int(0.8 * len(dataset))
val_size   = int(0.1 * len(dataset))
test_size  = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE)

print(f"✅ Dataset ready! Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")


# =====================================================
# 7️⃣ Train Setup
# =====================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SketchRNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# =====================================================
# 8️⃣ Training Loop with Early Stopping
# =====================================================
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for x, _ in train_loader:
        x = x.to(device)
        optimizer.zero_grad()
        recon = model(x)
        loss = criterion(recon, x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    avg_train_loss = train_loss / len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, _ in val_loader:
            x = x.to(device)
            recon = model(x)
            loss = criterion(recon, x)
            val_loss += loss.item() * x.size(0)
    avg_val_loss = val_loss / len(val_loader.dataset)

    print(f"📊 Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_sketch_rnn.pth")
        print("💾 Saved new best model.")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

    if avg_val_loss <= TARGET_LOSS:
        print(f"🏁 Target loss {TARGET_LOSS} reached.")
        break

print("✅ Training complete.")


In [ ]:
# SketchRNN

In [3]:
# =====================================================
# 6️⃣ SketchRNN Classifier (instead of Autoencoder)
# =====================================================
class SketchRNNClassifier(nn.Module):
    def __init__(self, num_classes=len(categories)):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=2, padding=1),  # -> 14x14
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # -> 7x7
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU()
        )
        self.fc = nn.Linear(256, num_classes)  # classification head

    def forward(self, x):
        x = self.encoder(x)
        x = self.fc(x)
        return x


In [5]:
model = SketchRNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [6]:
for epoch in range(EPOCHS):
    # ---- Train ----
    model.train()
    train_loss, correct, total = 0, 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)

        _, predicted = outputs.max(1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_acc = correct / total

    # ---- Validation ----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            val_loss += loss.item() * x.size(0)

            _, predicted = outputs.max(1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = correct / total

    print(f"📊 Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}")


📊 Epoch 1/50 | Train Loss: 1.6073, Acc: 0.5721 | Val Loss: 1.1494, Acc: 0.6839
📊 Epoch 2/50 | Train Loss: 1.0162, Acc: 0.7214 | Val Loss: 0.9856, Acc: 0.7334
📊 Epoch 3/50 | Train Loss: 0.8391, Acc: 0.7646 | Val Loss: 0.9425, Acc: 0.7379
📊 Epoch 4/50 | Train Loss: 0.7134, Acc: 0.7963 | Val Loss: 0.9278, Acc: 0.7452
📊 Epoch 5/50 | Train Loss: 0.6088, Acc: 0.8220 | Val Loss: 0.9571, Acc: 0.7463
📊 Epoch 6/50 | Train Loss: 0.5088, Acc: 0.8501 | Val Loss: 0.9873, Acc: 0.7455
📊 Epoch 7/50 | Train Loss: 0.4211, Acc: 0.8714 | Val Loss: 1.0758, Acc: 0.7407
📊 Epoch 8/50 | Train Loss: 0.3400, Acc: 0.8952 | Val Loss: 1.1704, Acc: 0.7325
📊 Epoch 9/50 | Train Loss: 0.2750, Acc: 0.9123 | Val Loss: 1.3017, Acc: 0.7306
📊 Epoch 10/50 | Train Loss: 0.2225, Acc: 0.9278 | Val Loss: 1.4354, Acc: 0.7285
📊 Epoch 11/50 | Train Loss: 0.1822, Acc: 0.9410 | Val Loss: 1.5613, Acc: 0.7200
📊 Epoch 12/50 | Train Loss: 0.1493, Acc: 0.9511 | Val Loss: 1.7462, Acc: 0.7217
📊 Epoch 13/50 | Train Loss: 0.1292, Acc: 0.9575 |

In [7]:
# =====================================================
# 9️⃣ Evaluate on Test Set
# =====================================================
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        _, predicted = outputs.max(1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

test_acc = correct / total
print(f"✅ Test Accuracy: {test_acc:.4f}")


✅ Test Accuracy: 0.7192


In [1]:
# =====================================================
# 1️⃣ Imports
# =====================================================
import os
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models

# =====================================================
# 2️⃣ Configurations
# =====================================================
DATA_DIR = "quickdraw_npy"
SAMPLES_PER_CLASS = 8000   # ViT expects 224x224
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20
LR = 1e-4
NUM_CLASSES = 51
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

categories = [
    "airplane","ant","apple","backpack","banana","bed","bicycle","bird","book","bread",
    "bus","cake","camera","car","cat","chair","clock","computer","cup","dog","door",
    "duck","envelope","eye","face","fish","flower","fork","frog","guitar","hat","horse",
    "house","lollipop","pencil","pig","pizza","rabbit","shoe","snake","spider","spoon",
    "star","sun","train","tree","truck","umbrella","alarm_clock","birthday_cake","butterfly"
]

# =====================================================
# 3️⃣ Dataset
# =====================================================
class SketchDataset(Dataset):
    def __init__(self, root_dir, categories, samples_per_class, transform=None):
        self.data, self.labels = [], []
        self.transform = transform
        for label, cat in enumerate(categories):
            file_path = os.path.join(root_dir, f"{cat}.npy")
            drawings = np.load(file_path)
            drawings = drawings.reshape(-1, 28, 28)
            if len(drawings) < samples_per_class:
                continue
            idx = np.random.choice(len(drawings), samples_per_class, replace=False)
            self.data.append(drawings[idx])
            self.labels.extend([label] * samples_per_class)

        self.data = np.vstack(self.data).astype(np.uint8)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        img = np.stack([img]*3, axis=-1)  # convert grayscale -> 3 channels
        if self.transform:
            img = self.transform(img)
        y = self.labels[idx]
        return img, y

# =====================================================
# 4️⃣ Data transforms (resize + normalization for ViT)
# =====================================================
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

dataset = SketchDataset(DATA_DIR, categories, SAMPLES_PER_CLASS, transform=transform)
train_size = int(0.8 * len(dataset))
val_size   = int(0.1 * len(dataset))
test_size  = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_data, batch_size=BATCH_SIZE, num_workers=2)
test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=2)

print(f"✅ Dataset ready! Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")

# =====================================================
# 5️⃣ Vision Transformer Model
# =====================================================
model = models.vit_b_16(weights="IMAGENET1K_V1")   # Pretrained
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)

# =====================================================
# 6️⃣ Training Loop
# =====================================================
def train_model(model, train_loader, val_loader, epochs):
    best_val_acc = 0
    for epoch in range(epochs):
        # ---- Train ----
        model.train()
        train_loss, correct, total = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * x.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

        train_acc = correct / total
        avg_train_loss = train_loss / len(train_loader.dataset)

        # ---- Validation ----
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                outputs = model(x)
                loss = criterion(outputs, y)
                val_loss += loss.item() * x.size(0)
                _, predicted = outputs.max(1)
                correct += (predicted == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        avg_val_loss = val_loss / len(val_loader.dataset)

        print(f"📊 Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Acc: {val_acc:.4f}")

        # save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_vit.pth")
            print("💾 Saved new best model.")

    print("✅ Training complete.")

# Train
train_model(model, train_loader, val_loader, EPOCHS)

# =====================================================
# 7️⃣ Evaluate on Test Set
# =====================================================
model.load_state_dict(torch.load("best_vit.pth"))  # load best checkpoint
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        outputs = model(x)
        _, predicted = outputs.max(1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

test_acc = correct / total
print(f"🎯 Test Accuracy (ViT): {test_acc:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'quickdraw_npy/airplane.npy'